In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.compose import TransformedTargetRegressor

from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

In [2]:
data = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
data.columns = [x.strip() for x in data.columns.tolist()]
X = data.drop(columns = ['weekday_is_monday',
 'weekday_is_saturday',
 'LDA_00',
 'n_unique_tokens',
 'n_non_stop_words',
 'self_reference_avg_sharess',
 'rate_positive_words',
 'LDA_04',
 'LDA_02',
 'kw_max_max',
 'kw_avg_min',
 'kw_avg_avg',
 'rate_negative_words',
 'min_negative_polarity', 'url', 'timedelta', 'shares'])
y = data['shares']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

# Random Forest Model

In [10]:
random_forest_model = RandomForestRegressor(max_depth = 20, bootstrap = True, max_features = 10, n_estimators = 400, 
                                            oob_score = True)
random_forest_model = TransformedTargetRegressor(random_forest_model, func = np.log, 
                                                 inverse_func = np.exp).fit(X_train, y_train)

RandomForestRegressor(max_depth=20, max_features=10, n_estimators=400,
                      oob_score=True)

In [31]:
mean_absolute_error(y_test, new_model.predict(X_test))

2271.083448056759

# XGBoost

In [10]:
xgb_model = xgb.XGBRegressor(random_state = 1, max_depth = 5, learning_rate = 0.001,
                        n_estimators = 8290,reg_lambda = 0, reg_alpha = 10, gamma = 14, subsample = 0.75, 
                         colsample_bytree = 0.5, eval_metric = mean_absolute_error)
xgb_model = TransformedTargetRegressor(xgb_model, func = np.log, inverse_func = np.exp)

xgb_model.fit(X_train, y_train)

TransformedTargetRegressor(func=<ufunc 'log'>, inverse_func=<ufunc 'exp'>,
                           regressor=XGBRegressor(base_score=None, booster=None,
                                                  callbacks=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=0.5,
                                                  early_stopping_rounds=None,
                                                  enable_categorical=False,
                                                  eval_metric=<function mean_absolute_error at 0x000002109BFF43A0>,
                                                  feature_types=None, gamma=14,
                                                  gpu_id=None, grow_policy=None,
                                                  importance_type=None,
                                                  interaction_constraints=None,
                                                  learning_rate=0.001,
                                                  max_bin=None,
                                                  max_cat_threshold=None,
                                                  max_cat_to_onehot=None,
                                                  max_delta_step=None,
                                                  max_depth=5, max_leaves=None,
                                                  min_child_weight=None,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  n_estimators=8290,
                                                  n_jobs=None,
                                                  num_parallel_tree=None,
                                                  predictor=None,
                                                  random_state=1, ...))

In [11]:
mean_absolute_error(y_test, xgb_model.predict(X_test))

2273.104780867085

# Mars

In [35]:
mars_model = Earth(max_terms = 500, max_degree = 2)
mars_model = TransformedTargetRegressor(mars_model, func = np.log, inverse_func = np.exp).fit(X_train, y_train)

C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [36]:
mean_absolute_error(y_test, mars_model.predict(X_test))

2285.0805519733763

In [45]:
#Voting ensemble: Averaging the predictions of all models
en=VotingRegressor(estimators = [('xgb',xgb_model),('random_forest',random_forest_model),('mars',mars_model)])
en.fit(X_train,y_train)
print("Ensemble model MAE = ", mean_absolute_error(en.predict(X_test),y_test))

C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Ensemble model MAE =  2457.031171539462


In [46]:
#Stacking using LinearRegression as the metamodel
en2 = StackingRegressor(estimators = [('xgb', xgb_model),('random_forest', random_forest_model),('mars', mars_model)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en2.fit(X_train,y_train)
print("Linear regression metamodel MAE = ", mean_absolute_error(en.predict(X_test),y_test))

C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\nhj4\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


KeyboardInterrupt: 